In [2]:
# crown_from_full_wave.py
# Dual‑tangent circular crown inside a quarter rectangle.
# Inputs: full‑wave height & width (H_full, W_full), strut width w, R rule (R = R_factor * w).
# Outputs: centerline chord/sagitta, arm offsets, angles, and outer sagitta.

import math
from typing import Dict, Optional

def _solve_delta_for_quarter(H: float, W: float, w: float, R: float) -> float:
    """
    Solve for the contact angle δ (radians) for the quarter-rectangle construction:

        tan δ = 2 * (H - w/2 - R*(1 - cos δ)) / (W - 2*R*sin δ)

    Robust bisection on δ ∈ (5°, 89.9°), with coarse bracketing.
    """
    p = 0.5 * w
    A = H - p

    def F(d: float) -> float:
        s = math.sin(d)
        c = math.cos(d)
        denom = W - 2.0 * R * s
        if abs(denom) < 1e-14:
            denom = 1e-14 if denom >= 0 else -1e-14
        return math.tan(d) - (2.0 * (A - R * (1.0 - c))) / denom

    lo, hi = math.radians(5.0), math.radians(89.9)
    step = math.radians(0.05)

    # coarse scan to find a sign change or best point
    best_d = lo
    best_val = abs(F(lo))
    prev_d = lo
    prev_f = F(prev_d)
    bracket = None

    d = lo + step
    while d <= hi + 1e-12:
        fd = F(d)
        if abs(fd) < best_val:
            best_val, best_d = abs(fd), d
        if prev_f * fd < 0.0:
            bracket = (prev_d, d)
            break
        prev_d, prev_f = d, fd
        d += step

    if bracket is None:
        span = math.radians(5.0)
        lo = max(math.radians(1e-6), best_d - 0.5 * span)
        hi = min(math.radians(89.9), best_d + 0.5 * span)
    else:
        lo, hi = bracket

    # bisection
    flo, fhi = F(lo), F(hi)
    for _ in range(120):
        mid = 0.5 * (lo + hi)
        fm = F(mid)
        if abs(fm) < 1e-14 or (hi - lo) < 1e-12:
            return mid
        if flo * fm < 0.0:
            hi, fhi = mid, fm
        else:
            lo, flo = mid, fm

    return 0.5 * (lo + hi)


def quarter_wave_from_rect(
    rect_height_mm: float,
    rect_width_mm: float,
    strut_width_mm: float,
    R_factor: float = 2.5,
    R_override_mm: Optional[float] = None,
) -> Dict[str, float]:
    """
    Solve the crown geometry in a *quarter* rectangle (H_box, W_box).
    Circle center is at (W/2, w/2 + R), arc tangent to both straight arms.

    Returns (all mm/deg; centerline unless noted):
      {
        "delta_deg", "theta_deg",
        "X_mm", "Y_mm",           # arm offsets from the two quarter-rectangle corners to the contact
        "y_chord_mm",             # chord height above bottom
        "chord_mm", "sagitta_mm", # centerline chord & sagitta
        "Rc_mm",                  # centerline radius used
        "outer_sagitta_mm"        # outer-track sagitta (fold-lock check)
      }
    """
    H = float(rect_height_mm)
    W = float(rect_width_mm)
    w = float(strut_width_mm)
    R = float(R_override_mm) if R_override_mm is not None else float(R_factor) * w
    if R <= 0.0:
        raise ValueError("Centerline radius must be positive.")

    # Solve for delta
    delta = _solve_delta_for_quarter(H, W, w, R)

    s, c = math.sin(delta), math.cos(delta)
    # contact point horizontal offset from each corner
    X = 0.5 * (W - 2.0 * R * s)
    # chord height above bottom
    y_ch = (0.5 * w) + R * (1.0 - c)
    # centerline sagitta & chord
    h = R * (1.0 - c)
    chord = 2.0 * R * s
    # vertical arm
    Y = H - y_ch

    if h <= 0 or chord <= 0 or X < 0 or Y < 0:
        raise ValueError("Infeasible geometry for given (H_box, W_box, w, R).")

    return {
        "delta_deg": math.degrees(delta),
        "theta_deg": 2.0 * math.degrees(delta),
        "X_mm": X,
        "Y_mm": Y,
        "y_chord_mm": y_ch,
        "chord_mm": chord,
        "sagitta_mm": h,
        "Rc_mm": R,
        "outer_sagitta_mm": (R + 0.5 * w) * (1.0 - c),
    }


def crown_from_full_wave(
    H_full_mm: float,
    W_full_mm: float,
    strut_width_mm: float,
    R_factor: float = 2.5,
    R_override_mm: Optional[float] = None,
) -> Dict[str, float]:
    """
    Convenience wrapper: accepts *full-wave* height & width.
    Internally halves to a quarter-rectangle and calls quarter_wave_from_rect().
    """
    Hq = 0.5 * float(H_full_mm)  # quarter height
    Wq = 0.5 * float(W_full_mm)  # quarter width
    out = quarter_wave_from_rect(
        rect_height_mm=Hq,
        rect_width_mm=Wq,
        strut_width_mm=strut_width_mm,
        R_factor=R_factor,
        R_override_mm=R_override_mm,
    )
    # Add echoes of full-wave inputs for traceability
    out.update({"H_full_mm": float(H_full_mm), "W_full_mm": float(W_full_mm)})
    return out


# -----------------------
# Example usage / sanity:
# -----------------------
if __name__ == "__main__":
    # Your example: full-wave H=1.27 mm, W=0.708 mm
    print("Ends (w=0.060, R=2.5*w):")
    res_end = crown_from_full_wave(1.278968, 0.706858, 0.060, R_factor=2.5)
    for k, v in res_end.items():
        print(f"  {k:>18}: {v:.6f}" if isinstance(v, (int, float)) else f"  {k:>18}: {v}")

    print("\nBody (w=0.050, R=2.5*w):")
    res_body = crown_from_full_wave(1.278968, 0.706858, 0.050, R_factor=2.5)
    for k, v in res_body.items():
        print(f"  {k:>18}: {v:.6f}" if isinstance(v, (int, float)) else f"  {k:>18}: {v}")


Ends (w=0.060, R=2.5*w):
           delta_deg: 86.703455
           theta_deg: 173.406910
                X_mm: 0.026963
                Y_mm: 0.468110
          y_chord_mm: 0.171374
            chord_mm: 0.299504
          sagitta_mm: 0.141374
               Rc_mm: 0.150000
    outer_sagitta_mm: 0.169649
           H_full_mm: 1.278968
           W_full_mm: 0.706858

Body (w=0.050, R=2.5*w):
           delta_deg: 84.047469
           theta_deg: 168.094938
                X_mm: 0.052388
                Y_mm: 0.502447
          y_chord_mm: 0.137037
            chord_mm: 0.248652
          sagitta_mm: 0.112037
               Rc_mm: 0.125000
    outer_sagitta_mm: 0.134444
           H_full_mm: 1.278968
           W_full_mm: 0.706858
